In [1]:
import pandas as pd

In [8]:
df = pd.read_csv('/content/bio_anomalies_false.csv')

In [9]:
df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,01-03-2025,jammu and kashmir,punch,185101,643,1091
1,01-03-2025,gujarat,sabarkantha,383440,56,91
2,01-03-2025,andhra pradesh,warangal,506163,39,92
3,01-03-2025,nagaland,mokokchung,798613,18,70
4,01-03-2025,jammu and kashmir,leh,194401,52,43


In [10]:
df.shape

(243289, 6)

In [11]:
pip install rapidfuzz jellyfish

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 27.6 MB/s eta 0:00:00


In [12]:
## new

import json
import numpy as np
import pandas as pd
from rapidfuzz import process, fuzz
import jellyfish

with open("/content/state_district_reference.json", "r") as f:
    ref = json.load(f)

state_districts = {
    state: data["districts"]
    for state, data in ref.items()
}

def enhanced_similarity(a, b, **kwargs):
    return max(
        fuzz.token_set_ratio(a, b),
        fuzz.partial_ratio(a, b),
        fuzz.ratio(a, b),
        jellyfish.jaro_winkler_similarity(a, b) * 100
    )

def token_or_substring_ok(a, b):
    a = a.lower()
    b = b.lower()

    a_tokens = a.split()
    b_tokens = b.split()

    # 1️⃣ token overlap (multi-word safety)
    if set(a_tokens) & set(b_tokens):
        return True

    # 2️⃣ substring
    if a in b or b in a:
        return True

    # 3️⃣ single-word prefix rejection
    if len(a_tokens) == 1 and len(b_tokens) == 1:
        if a[:4] != b[:4]:
            return False

    # 4️⃣ character overlap
    a_chars = set(a)
    b_chars = set(b)
    overlap_ratio = len(a_chars & b_chars) / min(len(a_chars), len(b_chars))

    return overlap_ratio >= 0.6



def best_district_match(state, district, threshold=80, min_gap=12):
    if pd.isna(state) or pd.isna(district):
        return np.nan

    if state not in state_districts:
        return np.nan

    matches = process.extract(
        district,
        state_districts[state],
        scorer=enhanced_similarity,
        limit=2
    )

    if not matches:
        return np.nan

    best_name, best_score, _ = matches[0]

    if best_score < threshold:
        return np.nan

    if len(matches) > 1:
        if (best_score - matches[1][1]) < min_gap:
            return np.nan

    if not token_or_substring_ok(district, best_name):
        return np.nan

    return best_name



df["district_new"] = df.apply(
    lambda x: best_district_match(x["state"], x["district"]),
    axis=1
)


In [13]:
df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_,district_new
0,01-03-2025,jammu and kashmir,punch,185101,643,1091,NaN
1,01-03-2025,gujarat,sabarkantha,383440,56,91,sabar kantha
2,01-03-2025,andhra pradesh,warangal,506163,39,92,NaN
3,01-03-2025,nagaland,mokokchung,798613,18,70,mokokchung
4,01-03-2025,jammu and kashmir,leh,194401,52,43,NaN


In [14]:
#Saving Cleaned Data
df_cleaned = df[~df['district_new'].isna()]

In [15]:
df_cleaned.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_,district_new
1,01-03-2025,gujarat,sabarkantha,383440,56,91,sabar kantha
3,01-03-2025,nagaland,mokokchung,798613,18,70,mokokchung
6,01-03-2025,nagaland,dimapur,797103,155,381,dimapur
11,01-03-2025,nagaland,phek,797108,29,112,phek
14,01-03-2025,madhya pradesh,khargone,451442,335,427,khargone (west nimar)


In [16]:
df_cleaned['district'] = df_cleaned['district_new']

/tmp/ipython-input-1763447331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['district'] = df_cleaned['district_new']


In [18]:
df_cleaned.drop(columns=['district_new'], inplace=True)


/tmp/ipython-input-3414518825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop(columns=['district_new'], inplace=True)


In [19]:
df_cleaned.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_
1,01-03-2025,gujarat,sabar kantha,383440,56,91
3,01-03-2025,nagaland,mokokchung,798613,18,70
6,01-03-2025,nagaland,dimapur,797103,155,381
11,01-03-2025,nagaland,phek,797108,29,112
14,01-03-2025,madhya pradesh,khargone (west nimar),451442,335,427


In [34]:
df_cleaned.shape

(73429, 6)

In [27]:
df_cleaned.to_csv('bio_anomalies_anomalies_clean.csv',index=False)

In [28]:
# Saving False Data
df_false = df[df['district_new'].isna()]

In [33]:
df_false.shape

(169860, 7)

In [30]:
df_false.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_,district_new
0,01-03-2025,jammu and kashmir,punch,185101,643,1091,NaN
2,01-03-2025,andhra pradesh,warangal,506163,39,92,NaN
4,01-03-2025,jammu and kashmir,leh,194401,52,43,NaN
5,01-03-2025,west bengal,barddhaman,713158,12,13,NaN
7,01-03-2025,delhi,south west delhi,110072,183,153,NaN


In [31]:
#simple check
df_false[~df_false['district_new'].isna()]

,date,state,district,pincode,bio_age_5_17,bio_age_17_,district_new


In [35]:
df_false.to_csv('bio_anomalies_anomalies_false.csv',index=False)